In [198]:
import os
import pandas as pd
import numpy as np
from pyspark.ml import Pipeline
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from sklearn.metrics import confusion_matrix
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder
import numpy as np
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [199]:
spark = SparkSession.builder.master("local[2]").appName("Spaceship_rescue").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(spark.sparkContext)
spark

c:\Users\jarno\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [200]:
train_data = 'train_space.csv'
test_data = 'test_space.csv'
df_train = spark.read.csv(train_data, header=True, inferSchema=True)
df_test = spark.read.csv(test_data, header=True, inferSchema=True)

In [201]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType, DoubleType, IntegerType
df_train = df_train.withColumn("CryoSleep",col("CryoSleep").cast(BooleanType()).cast(IntegerType()))\
    .withColumn("VIP",col("VIP").cast(BooleanType()).cast(IntegerType()))\
    .withColumn("Transported", col("Transported").cast(BooleanType()).cast(IntegerType()))



In [202]:
# fill null values with median
from pyspark.ml.feature import Imputer
imputer_cols = ["Age", "ShoppingMall", "VRDeck"]

imputer = Imputer(inputCols=imputer_cols, outputCols=imputer_cols).setStrategy("median")
df_train = imputer.fit(df_train).transform(df_train)
df_test = imputer.fit(df_test).transform(df_test)


In [203]:
# Deze ga ik niet gebruiken
cols = ('Cabin','RoomService', 'FoodCourt', 'Spa', 'Name', 'PassengerId')
df_train = df_train.drop(*cols)
cols = ('Cabin','RoomService', 'FoodCourt', 'Spa', 'Name')
df_test = df_test.drop(*cols) # passengerid hebk straks nodig voor de submission

# drop all rows with null values
df_train = df_train.na.drop()
df_test = df_test.na.drop()


In [204]:
# change HomePlanet columns that starat with 'Ear' to 0 
df_train = df_train.withColumn("HomePlanet", when(col("HomePlanet").startswith("Ear"), 0).otherwise(col("HomePlanet")))
df_test = df_test.withColumn("HomePlanet", when(col("HomePlanet").startswith("Ear"), 0).otherwise(col("HomePlanet")))
# change HomePlanet columns that starat with 'Mar' to 1
df_train = df_train.withColumn("HomePlanet", when(col("HomePlanet").startswith("Mar"), 1).otherwise(col("HomePlanet")))
df_test = df_test.withColumn("HomePlanet", when(col("HomePlanet").startswith("Mar"), 1).otherwise(col("HomePlanet")))
# change HomePlanet columns that starat with 'Europ ' to 2
df_train = df_train.withColumn("HomePlanet", when(col("HomePlanet").startswith("Europ"), 2).otherwise(col("HomePlanet")))
df_test = df_test.withColumn("HomePlanet", when(col("HomePlanet").startswith("Europ"), 2).otherwise(col("HomePlanet")))

# Change Destination that start with T to 0
df_train = df_train.withColumn("Destination", when(col("Destination").startswith("T"), 0).otherwise(col("Destination")))
df_test = df_test.withColumn("Destination", when(col("Destination").startswith("T"), 0).otherwise(col("Destination")))
# Change Destination that start with 5 to 1
df_train = df_train.withColumn("Destination", when(col("Destination").startswith("5"), 1).otherwise(col("Destination")))
df_test = df_test.withColumn("Destination", when(col("Destination").startswith("5"), 1).otherwise(col("Destination")))
# Change Destination that start with P to 2
df_train = df_train.withColumn("Destination", when(col("Destination").startswith("P"), 2).otherwise(col("Destination")))
df_test = df_test.withColumn("Destination", when(col("Destination").startswith("P"), 2).otherwise(col("Destination")))


# cast to integer
df_train = df_train.withColumn("HomePlanet",col("HomePlanet").cast(IntegerType()))\
    .withColumn("Destination",col("Destination").cast(IntegerType()))\

df_test = df_test.withColumn("HomePlanet",col("HomePlanet").cast(IntegerType()))\
    .withColumn("Destination",col("Destination").cast(IntegerType()))\
        


In [205]:
# nu nog de categorische waardes omzetten naar numerieke waardes
## Ging iets mis mee, probeer andere manier!, accuracy was altijd 100%
# label_cols = ["HomePlanet",  "Destination" ]
# from pyspark.ml.feature import StringIndexer
# # Label encoding
# # change all columns to strings
# for col in label_cols:
#     df_train = df_train.withColumn(col, df_train[col].cast(StringType()))
#     df_test = df_test.withColumn(col, df_test[col].cast(StringType()))
    
# indexers = [StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid="skip").fit(df_train) for column in label_cols ]
# pipeline = Pipeline(stages=indexers)
# df_train = pipeline.fit(df_train).transform(df_train)
# df_test = pipeline.fit(df_test).transform(df_test)

# # delete the old columns
# df_train = df_train.drop(*label_cols)
# df_test = df_test.drop(*label_cols)



In [206]:
df_train.show()

+----------+---------+-----------+----+---+------------+------+-----------+
|HomePlanet|CryoSleep|Destination| Age|VIP|ShoppingMall|VRDeck|Transported|
+----------+---------+-----------+----+---+------------+------+-----------+
|         2|        0|          0|39.0|  0|         0.0|   0.0|          0|
|         0|        0|          0|24.0|  0|        25.0|  44.0|          1|
|         2|        0|          0|58.0|  1|         0.0|  49.0|          0|
|         2|        0|          0|33.0|  0|       371.0| 193.0|          0|
|         0|        0|          0|16.0|  0|       151.0|   2.0|          1|
|         0|        0|          2|44.0|  0|         0.0|   0.0|          1|
|         0|        0|          0|26.0|  0|         3.0|   0.0|          1|
|         0|        1|          0|28.0|  0|         0.0|   0.0|          1|
|         0|        0|          0|35.0|  0|        17.0|   0.0|          1|
|         2|        1|          1|14.0|  0|         0.0|   0.0|          1|
|         2|

In [207]:
df_train = df_train.na.drop("any")
df_train.count()

df_train.show(1)

+----------+---------+-----------+----+---+------------+------+-----------+
|HomePlanet|CryoSleep|Destination| Age|VIP|ShoppingMall|VRDeck|Transported|
+----------+---------+-----------+----+---+------------+------+-----------+
|         2|        0|          0|39.0|  0|         0.0|   0.0|          0|
+----------+---------+-----------+----+---+------------+------+-----------+
only showing top 1 row



In [208]:
df_train = df_train.withColumnRenamed('Transported',"label")
features = df_train.columns[0:-1]

In [209]:


va = VectorAssembler(inputCols = features, outputCol='features')

va_df = va.transform(df_train)
va_df = va_df.select(['features', 'label'])
va_df.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(7,[0,3],[2.0,39.0])|    0|
|(7,[3,5,6],[24.0,...|    1|
|[2.0,0.0,0.0,58.0...|    0|
+--------------------+-----+
only showing top 3 rows



In [210]:
(train, test) = va_df.randomSplit([0.8, 0.2])

In [211]:
rfc = RandomForestClassifier(featuresCol="features", labelCol="label")

paramGrid = ParamGridBuilder() \
    .addGrid(rfc.numTrees, [int(x) for x in np.linspace(start = 10, stop = 50, num = 3)]) \
    .addGrid(rfc.maxDepth, [int(x) for x in np.linspace(start = 5, stop = 25, num = 3)]) \
    .build()

crossval = CrossValidator(estimator=rfc,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(),
                          numFolds=3)

cvModel = crossval.fit(train)
predictions = cvModel.transform(test)

In [212]:
evaluator=MulticlassClassificationEvaluator(predictionCol="prediction")
acc = evaluator.evaluate(predictions)
print("Prediction Accuracy: ", acc)

y_pred=predictions.select("prediction").collect()
y_orig=predictions.select("label").collect()

cm = confusion_matrix(y_orig, y_pred)
print("Confusion Matrix:")
print(cm)

Prediction Accuracy:  0.7339414588212347
Confusion Matrix:
[[598 179]
 [247 578]]


In [213]:
va = VectorAssembler(inputCols = features, outputCol='features')

va_df_test = va.transform(df_test)
va_df_test = va_df_test.select(['features'])

In [214]:
#cvModel = crossval.fit(train)
y_test_predictions = cvModel.transform(va_df_test)

In [215]:
import pandas as pd
df_test_cp = df_test.toPandas()
y_test_pred = y_test_predictions.toPandas()
data={"PassengerId":df_test_cp['PassengerId'],"Transported":y_test_pred['prediction'].astype('bool')}
data=pd.DataFrame(data)
data=data.to_csv('submission.csv',index=False)

In [216]:
pd.read_csv('./submission.csv')

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,False
4,0023_01,False
...,...,...
3920,9266_01,False
3921,9266_02,True
3922,9269_01,False
3923,9271_01,True
